![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/36.0.Loading_Medical_and_Open_Source_LLMs.ipynb)

# Loading Medical and Open Source LLMs



`LLMLoader` is designed to interact with a LLMs that are converted into gguf format. This module allows using John Snow Labs' licensed LLMs at various sizes that are finetuned on medical context for certain tasks. It provides various methods for setting parameters, loading models, generating text, and retrieving metadata. The `LLMLoader` includes methods for setting various parameters such as input prefix, suffix, cache prompt, number of tokens to predict, sampling techniques, temperature, penalties, and more. Overall, the `LLMLoader`  provides a flexible and extensible framework for interacting with language models in a Python and Scala environment using PySpark and Java.

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=True
nlp.install(refresh_install=True)

👌 Detected license file /content/5.4.0.spark_nlp_for_healthcare.json
🚨 Outdated OCR Secrets in license file. Version=5.1.2 but should be Version=5.4.0
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/5.4.0.spark_nlp_for_healthcare.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-5.4.0-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==5.4.0 installed! ✅ Heal the planet with NLP! 


In [ ]:
from johnsnowlabs import nlp, medical
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/5.4.0.spark_nlp_for_healthcare.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.4.0, 💊Spark-Healthcare==5.4.0, running on ⚡ PySpark==3.4.0


In [ ]:
spark

# Medical LLMs



| Model Name              | Description |
|-------------------------|-------------|
|[JSL_MedS_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q16_v1_en.html)      | Summarization and Q&A  |
|[JSL_MedS_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q8_v1_en.html)       | Summarization and Q&A |
|[JSL_MedS_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q4_v1_en.html)       | Summarization and Q&A  |
|[JSL_MedM_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q16_v1_en.html)      |  Summarization, Q&A, RAG, and Chat |
|[JSL_MedM_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q8_v1_en.html)       | Summarization, Q&A, RAG, and Chat |
|[JSL_MedM_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q4_v1_en.html)       | Summarization, Q&A, RAG, and Chat |
|[JSL_MedSNer_ZS_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zs_q16_v1_en.html)| Extract and link medical named entities |
|[JSL_MedSNer_ZS_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zs_q8_v1_en.html) | Extract and link medical named entities |
|[JSL_MedSNer_ZS_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zs_q4_v1_en.html) | Extract and link medical named entities |


**We recommend using 8b quantized versions of the models as the qualitative performance difference between q16 and q8 versions is very negligible.**

## JSL_MedS


This LLM model is trained to perform Summarization and Q&A based on a given context.

In [ ]:
jsl_meds_llm = medical.LLMLoader(spark).pretrained("jsl_meds_q8_v1", "en", "clinical/models")

In [ ]:
prompt = """
Based on the following text, what age group is most susceptible to breast cancer?

## Text:
The exact cause of breast cancer is unknown. However, several risk factors can increase your likelihood of developing breast cancer, such as:
- A personal or family history of breast cancer
- A genetic mutation, such as BRCA1 or BRCA2
- Exposure to radiation
- Age (most commonly occurring in women over 50)
- Early onset of menstruation or late menopause
- Obesity
- Hormonal factors, such as taking hormone replacement therapy
"""

response = jsl_meds_llm.generate(prompt)

In [ ]:
response

' The age group most susceptible to breast cancer, as mentioned in the text, is women over the age of 50.'

## JSL_MedM


This LLM model is trained to perform Q&A, Summarization, RAG, and Chat.

In [ ]:
jsl_medm_llm = medical.LLMLoader(spark).pretrained("jsl_medm_q8_v1", "en", "clinical/models")

In [ ]:
prompt = """
A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.
Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""

response = jsl_medm_llm.generate(prompt)

In [ ]:
print(response)

The correct answer is E: Nitrofurantoin.

The patient is 22 weeks pregnant and has symptoms of burning upon urination, which is a common symptom of urinary tract infection (UTI). Nitrofurantoin is a first-line antibiotic for uncomplicated UTI in pregnant women.


In [ ]:
### Output:
"""The correct answer is E: Nitrofurantoin.

The patient is presenting with symptoms of urinary tract infection (UTI), which is common during pregnancy. Nitrofurantoin is a first-line antibiotic for uncomplicated UTI during pregnancy. It is safe and effective in treating UTI during pregnancy and has been used for many years without any adverse effects on the fetus.
"""

## JSL_MedSNer


This LLM model is trained to extract and link entities in a document.
Users needs to define an input schema as explained in the example section.
Drug is defined as a list which tells the model that there could be multiple drugs in the document and it has to extract all of them.
Each drug has properties like "name" and "reaction". Since "name" is only one, it is a string, but there could be multiple reactions, hence it is a list.
Similarly, users can define any schema for any type of entity.

In [ ]:
jsl_medner_llm = medical.LLMLoader(spark).pretrained("jsl_medsner_zs_q16_v1", "en", "clinical/models")

In [ ]:
prompt = """
### Template:
{
    "drugs": [
        {
            "name": "",
            "reactions": []
        }
    ]
}
### Text:
I feel a bit drowsy & have a little blurred vision , and some gastric problems .
I 've been on Arthrotec 50 for over 10 years on and off , only taking it when I needed it .
Due to my arthritis getting progressively worse , to the point where I am in tears with the agony.
Gp 's started me on 75 twice a day and I have to take it every day for the next month to see how I get on , here goes .
So far its been very good , pains almost gone , but I feel a bit weird , did n't have that when on 50.
"""

response = jsl_medner_llm.generate(prompt)

In [ ]:
response

' {\n    "drugs": [\n        {\n            "name": "Arthrotec",\n            "reactions": [\n                "drowsiness",\n                "blurred vision",\n                "gastric problems"\n            ]\n        }\n    ]\n}\n {\n    "drugs": [\n        {\n            "name": "Arthrotec",\n            "reactions": [\n                "drowsiness",\n                "blurred vision",\n                "gastric problems"\n            ]\n        }\n    ]\n}\n {\n    "drugs": [\n        {\n            "name": "Arthrotec",\n            "reactions": [\n                "drowsiness",\n                "blurred vision",\n                "gastric problems"\n            ]\n        }\n    ]\n}\n {\n    "drugs": [\n        {\n            "name": "Arthrotec",\n            "reactions": [\n                "drowsiness",\n                "blurred vision",\n                "gastric problems"\n            ]\n        }\n    ]\n}\n {\n    "drugs": [\n        {\n            "name": "Arthrotec",\n          

In [ ]:
print(response)

 {
    "drugs": [
        {
            "name": "Arthrotec",
            "reactions": [
                "drowsiness",
                "blurred vision",
                "gastric problems"
            ]
        }
    ]
}
 {
    "drugs": [
        {
            "name": "Arthrotec",
            "reactions": [
                "drowsiness",
                "blurred vision",
                "gastric problems"
            ]
        }
    ]
}
 {
    "drugs": [
        {
            "name": "Arthrotec",
            "reactions": [
                "drowsiness",
                "blurred vision",
                "gastric problems"
            ]
        }
    ]
}
 {
    "drugs": [
        {
            "name": "Arthrotec",
            "reactions": [
                "drowsiness",
                "blurred vision",
                "gastric problems"
            ]
        }
    ]
}
 {
    "drugs": [
        {
            "name": "Arthrotec",
            "reactions": [
                "drowsiness",
      

In [ ]:
####### Model: JSL_MedSNer_ZS_q16_v1
### Output:
"""
{
    "drugs": [
        {
            "name": "Arthrotec",
            "reactions": [
                "drowsy",
                "blurred vision",
                "gastric problems"
            ]
        }
    ]
}
"""

# Opensource LLM

## mistral-7b

In [ ]:
# ! pip install huggingface-hub

!huggingface-cli download TheBloke/Mistral-7B-v0.1-GGUF mistral-7b-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
mistral-7b-v0.1.Q4_K_M.gguf: 100% 4.37G/4.37G [01:17<00:00, 56.4MB/s]
Download complete. Moving file to mistral-7b-v0.1.Q4_K_M.gguf
mistral-7b-v0.1.Q4_K_M.gguf


In [ ]:
llm_loader = medical.LLMLoader(spark)

In [ ]:
%%time

llm_loader\
    .setUseChatTemplate(True)\
    .setTemperature(0.0)\
    .setStopStrings(["<|im_end|>"])\
    .encodeModel(
        "./mistral-7b-v0.1.Q4_K_M.gguf",
        "./mistral-7b-v0.1.Q4_K_M/",
        metadata = {
          "licensed": "false"
        })

CPU times: user 135 ms, sys: 18.8 ms, total: 154 ms
Wall time: 24.3 s


In [ ]:
!ls -l ./mistral-7b-v0.1.Q4_K_M/

total 4270088
-rw-r--r-- 1 root root 4372561920 Jul 22 11:19 gguf
-rw-r--r-- 1 root root        120 Jul 22 11:19 metadata.json


In [ ]:
%%time
llm = llm_loader.load("./mistral-7b-v0.1.Q4_K_M")

CPU times: user 6.2 ms, sys: 2.33 ms, total: 8.53 ms
Wall time: 763 ms


In [ ]:
%%time
prompt = "What is the indication for the drug Methadone?"
response = llm.generate(prompt)

CPU times: user 79.4 ms, sys: 8.88 ms, total: 88.3 ms
Wall time: 12.3 s


In [ ]:
response

'Methadone is used to treat opioid addiction. It is also used to treat severe pain.\n'